# Check ToM ROIs
Natalia Vélez, February 2022

In [ ]:
%matplotlib inline

import os, sys
import pandas as pd
import numpy as np
from nilearn import image,plotting
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join as opj

sys.path.append('..')
from utils import gsearch

sns.set_style('white')
sns.set_context('talk')

Make output directory:

In [ ]:
out_dir = 'outputs/roi_picker'
os.makedirs(out_dir, exist_ok=True)

Find ROI files:

In [ ]:
data_dir = '../../BIDS_data/derivatives/'
roi_dir = opj(data_dir, 'roi_picker')

roi_files = gsearch(roi_dir, '**', 'func', '*.nii.gz')
roi_files.sort()

print('Found %i ROI files' % len(roi_files))
print(*roi_files[:10], sep='\n')

## Tally functional ROIs

Tally files

In [ ]:
roi_df = pd.DataFrame({'path': roi_files})
roi_df['subject'] = roi_df.path.str.extract('(sub-[0-9]{2})')
roi_df['roi'] = roi_df.path.str.extract('(?<=desc-)([A-Z]+)')
roi_df.head()

Did all participants run successfully?

In [ ]:
roi_df['subject'].agg('nunique')

How many participants have each ROI?

In [ ]:
roi_df.groupby('roi').agg({'subject': 'nunique'})

Who's missing ROIs?

In [ ]:
roi_pivot = roi_df.groupby(['subject', 'roi']).size().unstack(fill_value=0).astype(bool)
roi_pivot[~roi_pivot.all(axis=1)]

## Check ROI location and extent

Plot locations and extent of each ROI:

In [ ]:
extent_list = []

for name, group in roi_df.groupby('roi'):
    # Load all ROI images
    roi_images = image.load_img(group.path.values.tolist())

    # Plot mean ROI
    mean_roi = image.mean_img(roi_images)
    mean_roi_plot = plotting.plot_glass_brain(mean_roi, colorbar=True, title=name)
    mean_roi_plot.savefig(opj(out_dir, '%s_location.png' % name))

    # Get ROI extent
    roi_extents = np.apply_over_axes(np.sum, roi_images.dataobj, range(roi_images.dataobj.ndim - 1)).flatten()
    roi_extents_df = group.copy()
    roi_extents_df['extent'] = roi_extents
    extent_list.append(roi_extents_df)

    fig,ax=plt.subplots()
    sns.distplot(roi_extents, rug=True, ax=ax)
    ax.set(title=name)
    plt.savefig(opj(out_dir, '%s_extent.png' % name))

Put together dataframe of ROI extents

In [ ]:
all_roi_extents = pd.concat(extent_list)
all_roi_extents.to_csv(opj(out_dir, 'all_roi_extents.csv'), index=False)
all_roi_extents.groupby('roi').agg({'extent': ['min', 'max', 'median']})